In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime
import math, statistics
import random

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
random.seed(2)
tf.random.set_seed(2)

In [ ]:
df_unstructure_full = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Project/CP2/full_unstructed_data.csv")

In [ ]:
df_unstructure_full

gamer_id  ...  y
0       000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  ...  0
1       000012d4f585a9bb5d6975e157b79b3538b04885ac19c6...  ...  1
2       00002a907f341c6113a96699f6da7d69bff34f29853c47...  ...  1
3       00003fb64d5509abbd5921dfd4736d9fc8d2c392a42c25...  ...  1
4       0000bbd7d65a8cc77af85023344fb46e3a00d3772dcbf3...  ...  0
...                                                   ...  ... ..
545907  fffeb0212f0f68f342421974ddf60b60b4468ca71f89fa...  ...  1
545908  ffff55ab88728761706a92ef75d68394c5dad8edd66818...  ...  1
545909  ffff84a159d96864ec6ee5ae6c9c4386d3060666bcc5a1...  ...  1
545910  ffffebdf9cff7c9d118dbb90fe145fbeb61c4978e1692d...  ...  1
545911  fffff1f88815e0ba807c3ff28ebcb94d9dbd8fce805eae...  ...  0

[545912 rows x 3 columns]

# Pad and encode texts

In [ ]:
# 문장별 단어 토큰화 : 1분 23초
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_unstructure_full['data'])
# tokens = [word_tokenize(sentence) for sentence in df_unstructure['day_cat_act']]

In [ ]:
tokenizer

In [ ]:
# sample
print('총 샘플의 개수 : {}'.format(len(df_unstructure_full['data'])))

# vocab size 계산
vocab_size = len(tokenizer.word_index) + 1
print('vocab size : {}'.format(vocab_size))

# 정수 인코딩
X_encoded = tokenizer.texts_to_sequences(df_unstructure_full['data'])
print('encoded text example : {}'.format(X_encoded[0]))

# max length 계산
max_len = max(len(i) for i in X_encoded)
print('max length : {}'.format(max_len))

총 샘플의 개수 : 545912
vocab size : 31951
encoded text example : [2, 334, 1, 84, 1, 18, 21, 5, 16, 43, 126, 433, 350, 31, 65, 14, 60, 219, 734, 791, 51, 104, 13, 20, 90, 327, 135, 36, 69, 10, 20, 90, 249, 135, 36, 69]
max length : 85


In [ ]:
# padding
X = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y = np.array(df_unstructure_full['y'])

In [ ]:
X.shape

(545912, 85)

In [ ]:
y.shape

(545912,)

# Load Word2Vec models

In [ ]:
# Saved model Load
wv_model_64_full = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab_Notebooks/Project/CP2/wv_model_64_full')
wv_model_128_full = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab_Notebooks/Project/CP2/wv_model_128_full')
wv_model_256_full = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab_Notebooks/Project/CP2/wv_model_256_full')

In [ ]:
# wv model vector shape
print(wv_model_64_full.vectors.shape)
print(wv_model_128_full.vectors.shape)
print(wv_model_256_full.vectors.shape)

(31955, 64)
(31955, 128)
(31955, 256)


# Create Embedding Matrix

In [ ]:
# embedding matrix shape

# dim = 64
embedding_matrix_64_full = np.zeros((vocab_size, 64))
print(np.shape(embedding_matrix_64_full))

# dim = 128
embedding_matrix_128_full = np.zeros((vocab_size, 128))
print(np.shape(embedding_matrix_128_full))

# dim = 256
embedding_matrix_256_full = np.zeros((vocab_size, 256))
print(np.shape(embedding_matrix_256_full))

(31951, 64)
(31951, 128)
(31951, 256)


In [ ]:
# 각 모델별 word vectors
def get_vector_64(word):
    if word in wv_model_64_full:
        return wv_model_64_full[word]
    else:
        return None

def get_vector_128(word):
    if word in wv_model_128_full:
        return wv_model_128_full[word]
    else:
        return None

def get_vector_256(word):
    if word in wv_model_256_full:
        return wv_model_256_full[word]
    else:
        return None

In [ ]:
# 각 모델별 embedding matrix 생성
for word, idx in tokenizer.word_index.items():
    temp = get_vector_64(word)
    if temp is not None:
        embedding_matrix_64_full[idx] = temp

for word, idx in tokenizer.word_index.items():
    temp = get_vector_128(word)
    if temp is not None:
        embedding_matrix_128_full[idx] = temp

for word, idx in tokenizer.word_index.items():
    temp = get_vector_256(word)
    if temp is not None:
        embedding_matrix_256_full[idx] = temp

In [ ]:
# index = 1 부터 임베딩 시작 ==> tokenizer.word_index 가 1부터 인덱스를 매기기 때문
shape_64 = np.shape(embedding_matrix_64_full)
shape_128 = np.shape(embedding_matrix_128_full)
shape_256 = np.shape(embedding_matrix_256_full)

print(shape_64)
print(shape_128)
print(shape_256)
# embedding_matrix[0] == 0 행렬

(31951, 64)
(31951, 128)
(31951, 256)


# CNN Modeling

In [ ]:
#  데이터분할 (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(436729, 85)
(109183, 85)
(436729,)
(109183,)


In [ ]:
num_filters = 100
kernel_size = 3
dropout_rate = 0.3
hidden_dim = 128
output_dim = 1
BATCH_SIZE = 512
NUM_EPOCHS = 100
VALID_SPLIT = 0.2


model64_full = Sequential()
Embed = Embedding(vocab_size, 64, weights=[embedding_matrix_64_full], input_length=max_len, trainable=False)
model64_full.add(Embed)
model64_full.add(Conv1D(num_filters, kernel_size=kernel_size, padding='valid', activation='relu'))
model64_full.add(GlobalMaxPooling1D())
model64_full.add(Dropout(dropout_rate))
model64_full.add(Dense(units=hidden_dim, activation='relu'))
model64_full.add(Dense(units=1, activation='sigmoid')) # 0.0~1.0 실수로 출력

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model64_full.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, verbose=0, mode='auto')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history64_full = model64_full.fit(X_train, y_train,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=VALID_SPLIT,
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/100
683/683 [==============================] - 16s 8ms/step - loss: 0.4900 - acc: 0.7479 - val_loss: 0.4108 - val_acc: 0.7892
Epoch 2/100
683/683 [==============================] - 5s 7ms/step - loss: 0.4113 - acc: 0.7876 - val_loss: 0.3810 - val_acc: 0.8045
Epoch 3/100
683/683 [==============================] - 5s 7ms/step - loss: 0.3924 - acc: 0.7976 - val_loss: 0.3666 - val_acc: 0.8132
Epoch 4/100
683/683 [==============================] - 5s 7ms/step - loss: 0.3798 - acc: 0.8050 - val_loss: 0.3599 - val_acc: 0.8157
Epoch 5/100
683/683 [==============================] - 5s 7ms/step - loss: 0.3718 - acc: 0.8100 - val_loss: 0.3550 - val_acc: 0.8205
Epoch 6/100
683/683 [==============================] - 5s 7ms/step - loss: 0.3661 - acc: 0.8127 - val_loss: 0.3492 - val_acc: 0.8227
Epoch 7/100
683/683 [==============================] - 5s 7ms/step - loss: 0.3623 - acc: 0.8148 - val_loss: 0.3489 - val_acc: 0.8234
Epoch 8/100
683/683 [==============================] - 5s 7ms/step -

# Predict

In [ ]:
# 확률값
y_pred64 = model64_full.predict(X_test)

In [ ]:
# 클래스
y_pred_class64 = np.around(y_pred64)

# CNN Modeling with embedding dimension 128

In [ ]:
num_filters = 100
kernel_size = 3
dropout_rate = 0.3
hidden_dim = 128
output_dim = 1
BATCH_SIZE = 512
NUM_EPOCHS = 100
VALID_SPLIT = 0.2


model128_full = Sequential()
Embed = Embedding(vocab_size, 128, weights=[embedding_matrix_128_full], input_length=max_len, trainable=False)
model128_full.add(Embed)
model128_full.add(Conv1D(num_filters, kernel_size=kernel_size, padding='valid', activation='relu'))
model128_full.add(GlobalMaxPooling1D())
model128_full.add(Dropout(dropout_rate))
model128_full.add(Dense(units=hidden_dim, activation='relu'))
model128_full.add(Dense(units=1, activation='sigmoid')) # 0.0~1.0 실수로 출력

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model128_full.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, verbose=0, mode='auto')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history128_full = model128_full.fit(X_train, y_train,
                       epochs=NUM_EPOCHS,
                       batch_size=BATCH_SIZE,
                       validation_split=VALID_SPLIT,
                       verbose=1,
                       callbacks=[early_stopping])

Epoch 1/100
683/683 [==============================] - 6s 8ms/step - loss: 0.4699 - acc: 0.7616 - val_loss: 0.4055 - val_acc: 0.7873
Epoch 2/100
683/683 [==============================] - 6s 8ms/step - loss: 0.3983 - acc: 0.7956 - val_loss: 0.3679 - val_acc: 0.8125
Epoch 3/100
683/683 [==============================] - 5s 8ms/step - loss: 0.3804 - acc: 0.8055 - val_loss: 0.3585 - val_acc: 0.8189
Epoch 4/100
683/683 [==============================] - 6s 8ms/step - loss: 0.3707 - acc: 0.8108 - val_loss: 0.3529 - val_acc: 0.8200
Epoch 5/100
683/683 [==============================] - 6s 8ms/step - loss: 0.3641 - acc: 0.8142 - val_loss: 0.3470 - val_acc: 0.8235
Epoch 6/100
683/683 [==============================] - 6s 8ms/step - loss: 0.3601 - acc: 0.8165 - val_loss: 0.3419 - val_acc: 0.8254
Epoch 7/100
683/683 [==============================] - 6s 8ms/step - loss: 0.3553 - acc: 0.8191 - val_loss: 0.3423 - val_acc: 0.8264
Epoch 8/100
683/683 [==============================] - 6s 8ms/step - 

# Predict

In [ ]:
# 확률값
y_pred128 = model128_full.predict(X_test)

In [ ]:
# 클래스
y_pred_class128 = np.around(y_pred128)

# CNN Modeling with embedding dimension 256

In [ ]:
num_filters = 100
kernel_size = 3
dropout_rate = 0.3
hidden_dim = 128
output_dim = 1
BATCH_SIZE = 512
NUM_EPOCHS = 100
VALID_SPLIT = 0.2


model256_full = Sequential()
Embed = Embedding(vocab_size, 256, weights=[embedding_matrix_256_full], input_length=max_len, trainable=False)
model256_full.add(Embed)
model256_full.add(Conv1D(num_filters, kernel_size=kernel_size, padding='valid', activation='relu'))
model256_full.add(GlobalMaxPooling1D())
model256_full.add(Dropout(dropout_rate))
model256_full.add(Dense(units=hidden_dim, activation='relu'))
model256_full.add(Dense(units=1, activation='sigmoid')) # 0.0~1.0 실수로 출력

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model256_full.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, verbose=0, mode='auto')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history256_full = model256_full.fit(X_train, y_train,
                       epochs=NUM_EPOCHS,
                       batch_size=BATCH_SIZE,
                       validation_split=VALID_SPLIT,
                       verbose=1,
                       callbacks=[early_stopping])

Epoch 1/100
683/683 [==============================] - 8s 11ms/step - loss: 0.4457 - acc: 0.7707 - val_loss: 0.3943 - val_acc: 0.7987
Epoch 2/100
683/683 [==============================] - 8s 12ms/step - loss: 0.3884 - acc: 0.8005 - val_loss: 0.3644 - val_acc: 0.8147
Epoch 3/100
683/683 [==============================] - 8s 11ms/step - loss: 0.3733 - acc: 0.8095 - val_loss: 0.3514 - val_acc: 0.8229
Epoch 4/100
683/683 [==============================] - 8s 11ms/step - loss: 0.3645 - acc: 0.8143 - val_loss: 0.3500 - val_acc: 0.8227
Epoch 5/100
683/683 [==============================] - 7s 11ms/step - loss: 0.3590 - acc: 0.8170 - val_loss: 0.3433 - val_acc: 0.8259
Epoch 6/100
683/683 [==============================] - 8s 11ms/step - loss: 0.3558 - acc: 0.8184 - val_loss: 0.3394 - val_acc: 0.8275
Epoch 7/100
683/683 [==============================] - 8s 12ms/step - loss: 0.3523 - acc: 0.8212 - val_loss: 0.3418 - val_acc: 0.8277
Epoch 8/100
683/683 [==============================] - 8s 12ms

# Predict

In [ ]:
# 확률값
y_pred256 = model256_full.predict(X_test)

In [ ]:
# 클래스
y_pred_class256 = np.around(y_pred256)

# Classification Report

In [ ]:
report = classification_report(y_test, y_pred_class64)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.77      0.80     46122
           1       0.84      0.89      0.86     63061

    accuracy                           0.84    109183
   macro avg       0.84      0.83      0.83    109183
weighted avg       0.84      0.84      0.83    109183



In [ ]:
report = classification_report(y_test, y_pred_class128)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.78      0.80     46122
           1       0.85      0.88      0.86     63061

    accuracy                           0.84    109183
   macro avg       0.83      0.83      0.83    109183
weighted avg       0.84      0.84      0.84    109183



In [ ]:
report = classification_report(y_test, y_pred_class256)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.79      0.81     46122
           1       0.85      0.87      0.86     63061

    accuracy                           0.84    109183
   macro avg       0.84      0.83      0.83    109183
weighted avg       0.84      0.84      0.84    109183



# Evaluate

In [ ]:
print("Evaluate on test data")
result64_full = model64_full.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result64_full)

Evaluate on test data
853/853 [==============================] - 3s 3ms/step - loss: 0.3252 - acc: 0.8360
test loss, test acc: [0.32516247034072876, 0.8359634876251221]


In [ ]:
print("Evaluate on test data")
result128_full = model128_full.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result128_full)

Evaluate on test data
853/853 [==============================] - 3s 3ms/step - loss: 0.3239 - acc: 0.8368
test loss, test acc: [0.3239174485206604, 0.8368152379989624]


In [ ]:
print("Evaluate on test data")
result256_full = model256_full.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result256_full)

Evaluate on test data
853/853 [==============================] - 3s 3ms/step - loss: 0.3192 - acc: 0.8393
test loss, test acc: [0.3191930949687958, 0.8392789959907532]
